In [161]:
# Supress Warnings
import warnings
warnings.filterwarnings('ignore')

# Visualization
import matplotlib.pyplot as plt
#import seaborn as sns

# Data Science
import numpy as np
import pandas as pd

# Multi-dimensional arrays and datasets
import xarray as xr

# Geospatial raster data handling
import rioxarray as rxr

# Geospatial data analysis
#import geopandas as gpd

# Geospatial operations
import rasterio
from rasterio import windows  
from rasterio import features  
from rasterio import warp
from rasterio.warp import transform_bounds 
from rasterio.windows import from_bounds 

# Image Processing
from PIL import Image

# Coordinate transformations
from pyproj import Proj, Transformer, CRS

# Feature Engineering
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

# Machine Learning
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
# Planetary Computer Tools
import pystac_client
import planetary_computer as pc
from pystac.extensions.eo import EOExtension as eo

# Others
import os
from tqdm import tqdm



In [3]:
# Open the GeoTIFF file
tiff_path = "Data/S2_median_fullBands_indeces.tiff"

# Read the bands from the GeoTIFF file
with rasterio.open(tiff_path) as src1:
    B01 = src1.read(1)  # Band [B01]
    B02 = src1.read(2)  # Band [B02]
    B03 = src1.read(3)  # Band [B03]
    B04 = src1.read(4)  # Band [B04]
    B05 = src1.read(5) #Band [B05]
    B06 = src1.read(6) # Band [B06]
    B07 = src1.read(7) # Band [B07]
    B08 = src1.read(8) # Band [B08]
    B8A = src1.read(9) # Band [B8A]
    B11 = src1.read(10) # Band [B11]
    B12 = src1.read(11) # Band [B12]
    ndwi = src1.read(12) #water index 
    ndvi = src1.read(13) #vegetation index
    ndbi = src1.read(14) #building index


In [4]:
# Extracts satellite band values from a GeoTIFF based on coordinates from a csv file and returns them in a DataFrame.

def map_satellite_data(tiff_path, csv_path):
    
    # Load the GeoTIFF data
    data = rxr.open_rasterio(tiff_path)
    tiff_crs = data.rio.crs

    # Read the Excel file using pandas
    df = pd.read_csv(csv_path)
    latitudes = df['Latitude'].values
    longitudes = df['Longitude'].values

    # 3. Convert lat/long to the GeoTIFF's CRS
    # Create a Proj object for EPSG:4326 (WGS84 - lat/long) and the GeoTIFF's CRS
    proj_wgs84 = Proj(init='epsg:4326')  # EPSG:4326 is the common lat/long CRS
    proj_tiff = Proj(tiff_crs)
    
    # Create a transformer object
    transformer = Transformer.from_proj(proj_wgs84, proj_tiff)

    B01_values = []
    B02_values = []
    B03_values = []
    B04_values = []
    B05_values = []
    B06_values = []
    B07_values = []
    B08_values = []
    B8A_values = []
    B11_values = []
    B12_values = []
    ndwi_values = []
    ndvi_values = []
    ndbi_values= []



# Iterate over the latitudes and longitudes, and extract the corresponding band values
    for lat, lon in tqdm(zip(latitudes, longitudes), total=len(latitudes), desc="Mapping values"):
    # Assuming the correct dimensions are 'y' and 'x' (replace these with actual names from data.coords)
    
        B01_value = data.sel(x=lon, y=lat,  band=1, method="nearest").values
        B01_values.append(B01_value)

        B02_value = data.sel(x=lon, y=lat,  band=2, method="nearest").values
        B02_values.append(B02_value)

        B03_value = data.sel(x=lon, y=lat,  band=3, method="nearest").values
        B03_values.append(B03_value)
    
        B04_value = data.sel(x=lon, y=lat, band=5, method="nearest").values
        B04_values.append(B04_value)

        B05_value = data.sel(x=lon, y=lat,  band=5, method="nearest").values
        B05_values.append(B05_value)
        
        B06_value = data.sel(x=lon, y=lat, band=6, method="nearest").values
        B06_values.append(B06_value)

        B07_value = data.sel(x=lon, y=lat,  band=7, method="nearest").values
        B07_values.append(B07_value)
    
        B08_value = data.sel(x=lon, y=lat, band=8, method="nearest").values
        B08_values.append(B08_value)

        B8A_value = data.sel(x=lon, y=lat, band=9, method="nearest").values
        B8A_values.append(B8A_value)

        B11_value = data.sel(x=lon, y=lat, band=10, method="nearest").values
        B11_values.append(B11_value)

        B12_value = data.sel(x=lon, y=lat, band=11, method="nearest").values
        B12_values.append(B12_value)

        ndwi_value = data.sel(x=lon, y=lat, band=12, method="nearest").values
        ndwi_values.append(ndwi_value)

        ndvi_value = data.sel(x=lon, y=lat, band=13, method="nearest").values
        ndvi_values.append(ndvi_value)

        ndbi_value = data.sel(x=lon, y=lat, band=14, method="nearest").values
        ndbi_values.append(ndbi_value)


    # Create a DataFrame with the band values
    # Create a DataFrame to store the band values
    df = pd.DataFrame()
    df['B01'] = B01_values
    df['B02'] = B02_values
    df['B03'] = B03_values
    df['B04'] = B04_values
    df['B05'] = B05_values
    df['B06'] = B06_values
    df['B07'] = B07_values
    df['B08'] = B08_values
    df['B8A'] = B8A_values
    df['B11'] = B11_values
    df['B12'] = B12_values
    df['ndwi'] = ndwi_values
    df['ndvi'] = ndvi_values
    df['ndbi'] = ndbi_values
    
    return df


### Sentinel-2 Bands Summary 
The following list of common bands can be loaded by the Open Data Cube (ODC) stac command.<br><br>
B01 = Coastal Aerosol = 60m <br>
B02 = Blue = 10m <br>
B03 = Green = 10m <br>
B04 = Red = 10m <br>
B05 = Red Edge (704 nm) = 20m <br>
B06 = Red Edge (740 nm) = 20m <br>
B07 = Red Edge (780 nm) = 20m <br>
B08 = NIR (833 nm) = 10m <br>
B8A = NIR (narrow 864 nm) = 20m <br>
B11 = SWIR (1.6 um) = 20m <br>
B12 = SWIR (2.2 um) = 20m

In [75]:
#this would be faster if I didn't import the indices from the tiff file and just redid them with pandas

UhiData_FileName = "/Users/jorgehernandez/Desktop/COS513/CourseProject/Code/Data/Training_data_uhi_index_2025-02-18.csv"
satellite_data =  "/Users/jorgehernandez/Desktop/COS513/CourseProject/Code/Data/S2_median_fullBands_indeces.tiff"

feature_data = map_satellite_data(satellite_data, UhiData_FileName)

Mapping values: 100%|██████████| 11229/11229 [02:14<00:00, 83.47it/s]


In [ ]:



#feature_data = median.to_dataframe().reset_index()
#There seems to be an error in which the index data is transferred wrong, all the indeces have the same values
#this is why I redo the columns here 
feature_data["ndwi"] = (feature_data["B03"] - feature_data["B08"])/(feature_data["B03"] + feature_data["B08"]) #Normalized Difference Water Index
feature_data["ndbi"] = (feature_data["B11"] - feature_data["B08"])/(feature_data["B11"] + feature_data["B08"]) # Normalized Difference Buildup Index
feature_data["ndvi"] = (feature_data["B08"] - feature_data["B04"])/(feature_data["B08"] + feature_data["B04"]) #Normalized Difference Vegetation Index


feature_data['ndvi'] = feature_data['ndvi'].replace([np.inf, -np.inf], np.nan) 
feature_data['ndwi'] = feature_data['ndwi'].replace([np.inf, -np.inf], np.nan) 
feature_data['ndbi'] = feature_data['ndbi'].replace([np.inf, -np.inf], np.nan) 


#nan_count = np.isnan(feature_data['ndwi'].to_numpy()).sum()
#print("Number of NaN values:", nan_count)  # Output: Number of NaN values: 1
#print(feature_data['ndwi'])


#Had to check manually that all elements in the indices where numeric. np.isinf and np.isnan dont work for some reason. 
#really annoying 
#for ele in feature_data['ndvi']:
 # print(ele)

In [87]:
def combine_two_datasets(dataset1,dataset2):
    '''
    Returns a  vertically concatenated dataset.
    Attributes:
    dataset1 - Dataset 1 to be combined 
    dataset2 - Dataset 2 to be combined
    '''
    
    data = pd.concat([dataset1,dataset2], axis=1)
    return data

In [94]:
# Combining ground data and feature data into a single dataset.
ground_df = pd.read_csv(UhiData_FileName)

uhi_data = combine_two_datasets(ground_df,feature_data)
uhi_data

,Longitude,Latitude,datetime,UHI Index,B01,B02,B03,B04,B05,B06,B07,B08,B8A,B11,B12,ndwi,ndvi,ndbi
0,-73.909167,40.813107,24-07-2021 15:53,1.030289,841.5,1053.0,1155.0,1481.5,1481.5,1660.5,1721.0,1832.0,1709.0,1792.0,1495.5,-0.226649,0.105779,-0.011038
1,-73.909187,40.813045,24-07-2021 15:53,1.030289,841.5,1053.0,1155.0,1481.5,1481.5,1660.5,1721.0,1832.0,1709.0,1792.0,1495.5,-0.226649,0.105779,-0.011038
2,-73.909215,40.812978,24-07-2021 15:53,1.023798,841.5,646.0,823.0,1130.5,1130.5,1883.0,2117.5,2241.0,2251.0,1548.0,1135.0,-0.462794,0.329379,-0.182898
3,-73.909242,40.812908,24-07-2021 15:53,1.023798,841.5,625.0,766.0,1130.5,1130.5,1883.0,2117.5,2200.0,2251.0,1548.0,1135.0,-0.483479,0.321123,-0.173959
4,-73.909257,40.812845,24-07-2021 15:53,1.021634,841.5,659.5,763.0,1077.5,1077.5,1783.0,2042.0,2161.0,2186.0,1617.5,1207.5,-0.478112,0.334568,-0.143840
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11224,-73.957050,40.790333,24-07-2021 15:57,0.972470,462.5,491.0,725.5,999.0,999.0,2612.0,3111.5,3152.0,3446.5,1925.0,1072.5,-0.625790,0.518670,-0.241678
11225,-73.957063,40.790308,24-07-2021 15:57,0.972470,462.5,506.5,741.5,999.0,999.0,2612.0,3111.5,3572.0,3446.5,1925.0,1072.5,-0.656196,0.562897,-0.299618
11226,-73.957093,40.790270,24-07-2021 15:57,0.981124,462.5,506.5,741.5,999.0,999.0,2612.0,3111.5,3572.0,3446.5,1925.0,1072.5,-0.656196,0.562897,-0.299618
11227,-73.957112,40.790253,24-07-2021 15:59,0.981245,462.5,506.5,741.5,999.0,999.0,2612.0,3111.5,3572.0,3446.5,1925.0,1072.5,-0.656196,0.562897,-0.299618


In [ ]:
# Combining ground data and feature data into a single dataset.
ground_df = pd.read_csv(UhiData_FileName)

uhi_data = combine_two_datasets(ground_df,feature_data)
uhi_data

,Longitude,Latitude,datetime,UHI Index,B01,B02,B03,B04,B05,B06,B07,B08,B8A,B11,B12,ndwi,ndvi,ndbi
0,-73.909167,40.813107,24-07-2021 15:53,1.030289,841.5,1053.0,1155.0,1481.5,1481.5,1660.5,1721.0,1832.0,1709.0,1792.0,1495.5,-0.226649,0.105779,-0.011038
1,-73.909187,40.813045,24-07-2021 15:53,1.030289,841.5,1053.0,1155.0,1481.5,1481.5,1660.5,1721.0,1832.0,1709.0,1792.0,1495.5,-0.226649,0.105779,-0.011038
2,-73.909215,40.812978,24-07-2021 15:53,1.023798,841.5,646.0,823.0,1130.5,1130.5,1883.0,2117.5,2241.0,2251.0,1548.0,1135.0,-0.462794,0.329379,-0.182898
3,-73.909242,40.812908,24-07-2021 15:53,1.023798,841.5,625.0,766.0,1130.5,1130.5,1883.0,2117.5,2200.0,2251.0,1548.0,1135.0,-0.483479,0.321123,-0.173959
4,-73.909257,40.812845,24-07-2021 15:53,1.021634,841.5,659.5,763.0,1077.5,1077.5,1783.0,2042.0,2161.0,2186.0,1617.5,1207.5,-0.478112,0.334568,-0.143840
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11224,-73.957050,40.790333,24-07-2021 15:57,0.972470,462.5,491.0,725.5,999.0,999.0,2612.0,3111.5,3152.0,3446.5,1925.0,1072.5,-0.625790,0.518670,-0.241678
11225,-73.957063,40.790308,24-07-2021 15:57,0.972470,462.5,506.5,741.5,999.0,999.0,2612.0,3111.5,3572.0,3446.5,1925.0,1072.5,-0.656196,0.562897,-0.299618
11226,-73.957093,40.790270,24-07-2021 15:57,0.981124,462.5,506.5,741.5,999.0,999.0,2612.0,3111.5,3572.0,3446.5,1925.0,1072.5,-0.656196,0.562897,-0.299618
11227,-73.957112,40.790253,24-07-2021 15:59,0.981245,462.5,506.5,741.5,999.0,999.0,2612.0,3111.5,3572.0,3446.5,1925.0,1072.5,-0.656196,0.562897,-0.299618


In [ ]:
# Combining ground data and feature data into a single dataset.
ground_df = pd.read_csv(UhiData_FileName)

uhi_data = combine_two_datasets(ground_df,feature_data)
uhi_data

,Longitude,Latitude,datetime,UHI Index,B01,B02,B03,B04,B05,B06,B07,B08,B8A,B11,B12,ndwi,ndvi,ndbi
0,-73.909167,40.813107,24-07-2021 15:53,1.030289,841.5,1053.0,1155.0,1481.5,1481.5,1660.5,1721.0,1832.0,1709.0,1792.0,1495.5,-0.226649,0.105779,-0.011038
1,-73.909187,40.813045,24-07-2021 15:53,1.030289,841.5,1053.0,1155.0,1481.5,1481.5,1660.5,1721.0,1832.0,1709.0,1792.0,1495.5,-0.226649,0.105779,-0.011038
2,-73.909215,40.812978,24-07-2021 15:53,1.023798,841.5,646.0,823.0,1130.5,1130.5,1883.0,2117.5,2241.0,2251.0,1548.0,1135.0,-0.462794,0.329379,-0.182898
3,-73.909242,40.812908,24-07-2021 15:53,1.023798,841.5,625.0,766.0,1130.5,1130.5,1883.0,2117.5,2200.0,2251.0,1548.0,1135.0,-0.483479,0.321123,-0.173959
4,-73.909257,40.812845,24-07-2021 15:53,1.021634,841.5,659.5,763.0,1077.5,1077.5,1783.0,2042.0,2161.0,2186.0,1617.5,1207.5,-0.478112,0.334568,-0.143840
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11224,-73.957050,40.790333,24-07-2021 15:57,0.972470,462.5,491.0,725.5,999.0,999.0,2612.0,3111.5,3152.0,3446.5,1925.0,1072.5,-0.625790,0.518670,-0.241678
11225,-73.957063,40.790308,24-07-2021 15:57,0.972470,462.5,506.5,741.5,999.0,999.0,2612.0,3111.5,3572.0,3446.5,1925.0,1072.5,-0.656196,0.562897,-0.299618
11226,-73.957093,40.790270,24-07-2021 15:57,0.981124,462.5,506.5,741.5,999.0,999.0,2612.0,3111.5,3572.0,3446.5,1925.0,1072.5,-0.656196,0.562897,-0.299618
11227,-73.957112,40.790253,24-07-2021 15:59,0.981245,462.5,506.5,741.5,999.0,999.0,2612.0,3111.5,3572.0,3446.5,1925.0,1072.5,-0.656196,0.562897,-0.299618


In [95]:
# Remove duplicate rows from the DataFrame based on specified columns and keep the first occurrence
columns_to_check = ['B01','B04','B06','B08','ndvi']
for col in columns_to_check:
    # Check if the value is a numpy array and has more than one dimension
    uhi_data[col] = uhi_data[col].apply(lambda x: tuple(x) if isinstance(x, np.ndarray) and x.ndim > 0 else x)

# Now remove duplicates
uhi_data = uhi_data.drop_duplicates(subset=columns_to_check, keep='first')
uhi_data.head()

uhi_data



,Longitude,Latitude,datetime,UHI Index,B01,B02,B03,B04,B05,B06,B07,B08,B8A,B11,B12,ndwi,ndvi,ndbi
0,-73.909167,40.813107,24-07-2021 15:53,1.030289,841.5,1053.0,1155.0,1481.5,1481.5,1660.5,1721.0,1832.0,1709.0,1792.0,1495.5,-0.226649,0.105779,-0.011038
2,-73.909215,40.812978,24-07-2021 15:53,1.023798,841.5,646.0,823.0,1130.5,1130.5,1883.0,2117.5,2241.0,2251.0,1548.0,1135.0,-0.462794,0.329379,-0.182898
3,-73.909242,40.812908,24-07-2021 15:53,1.023798,841.5,625.0,766.0,1130.5,1130.5,1883.0,2117.5,2200.0,2251.0,1548.0,1135.0,-0.483479,0.321123,-0.173959
4,-73.909257,40.812845,24-07-2021 15:53,1.021634,841.5,659.5,763.0,1077.5,1077.5,1783.0,2042.0,2161.0,2186.0,1617.5,1207.5,-0.478112,0.334568,-0.143840
6,-73.909312,40.812710,24-07-2021 15:53,1.015143,841.5,551.5,768.5,1077.5,1077.5,1783.0,2042.0,2472.0,2186.0,1617.5,1207.5,-0.525690,0.392872,-0.208950
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11219,-73.956957,40.790503,24-07-2021 15:57,0.974633,403.0,325.0,431.0,834.5,834.5,2607.5,3096.0,2671.0,3299.5,1807.0,898.5,-0.722115,0.523891,-0.192943
11220,-73.956973,40.790482,24-07-2021 15:57,0.976797,403.0,318.5,484.0,834.5,834.5,2607.5,3096.0,3051.0,3299.5,1807.0,898.5,-0.726167,0.570454,-0.256072
11223,-73.957038,40.790360,24-07-2021 15:57,0.972470,462.5,491.0,725.5,999.0,999.0,2612.0,3111.5,3152.0,3446.5,1925.0,1072.5,-0.625790,0.518670,-0.241678
11225,-73.957063,40.790308,24-07-2021 15:57,0.972470,462.5,506.5,741.5,999.0,999.0,2612.0,3111.5,3572.0,3446.5,1925.0,1072.5,-0.656196,0.562897,-0.299618


In [ ]:
#Building Neural Network 

#Check the scales of the features

#uhi_data_std = uhi_data
model1_data = uhi_data[['B01','B02', 'B03', 'B04', 'B05' , 'B08', 'B11','ndwi' , 'ndvi', 'ndbi' , 'UHI Index']]

#standardize the data in each column.
md1_data_std = model1_data.apply(lambda x: (x - x.mean()) / x.std(), axis=0) 
md1_data_std =md1_data_std.apply(lambda x: pd.to_numeric(x, errors = 'coerce'), axis=0) 


# Convert object columns to numeric
#for col in ['B02', 'B03', 'B05', 'B11']:
  #  model1_data[col] = pd.to_numeric(model1_data[col], errors='coerce')
for col in ['B02', 'B03', 'B05', 'B11']:
    na_count = md1_data_std[col].isna().sum()
    print(f"Number of NaN values in {col} after conversion: {na_count}")
    
print(md1_data_std.dtypes)

Number of NaN values in B02 after conversion: 0
Number of NaN values in B03 after conversion: 0
Number of NaN values in B05 after conversion: 0
Number of NaN values in B11 after conversion: 0
B01          float64
B02          float64
B03          float64
B04          float64
B05          float64
B08          float64
B11          float64
ndwi         float64
ndvi         float64
ndbi         float64
UHI Index    float64
dtype: object


In [208]:
#Divide data into training and testing
#md1_data_std = pd.to_numeric(md1_data_std)
X = md1_data_std.drop(columns=['UHI Index']).values
y = md1_data_std['UHI Index'].values


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2,random_state=123)
print(X_train.shape) 
print(y_test.shape)

(6683, 10)
(1671,)


In [ ]:

#Build neural network
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten

#Model 

model1 = Sequential([
    Dense(512 , input_shape=(10,), activation='relu'),
    Dense(128, activation = 'relu'),
    #Dense(30, activation = 'relu'),
    Dense(64 , activation = 'relu'),
    Dense(32, activation = 'relu'),
    Dense(16, activation = 'relu' ),
    Dense(1, activation = 'linear' ) #Output continous variable UHI index 
])


model1.compile(optimizer= 'RMSprop' , loss = "MSE" , metrics=[ 'mape'])

model1.fit(X_train, y_train, epochs=25) 





Epoch 1/25
209/209 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.9292 - mape: 174.0237
Epoch 2/25
209/209 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.8584 - mape: 193.1390
Epoch 3/25
209/209 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.8552 - mape: 187.8608
Epoch 4/25
209/209 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.8722 - mape: 212.7307
Epoch 5/25
209/209 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.8645 - mape: 188.9231
Epoch 6/25
209/209 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.8421 - mape: 194.5933
Epoch 7/25
209/209 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.8242 - mape: 207.7589
Epoch 8/25
209/209 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.8558 - mape: 193.0382
Epoch 9/25
209/209 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.8382 - mape: 213.1634
Epoch 10/25
209/209 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.8421 - mape: 196.8334
Epoch 11/25
209/209 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.8306 - mape: 200.8877
Epoch 12/25
209/209 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.8225 - mape

In [174]:
model1.evaluate(X_test,  y_test, verbose=2)


53/53 - 0s - 3ms/step - loss: 0.8643 - mape: 212.8611


[0.864333987236023, 212.86114501953125]

In [ ]:
y_lazy_pred = np.mean(y_test)*np.ones(len(y_test))

#print(mean_squared_error(y_test , y_lazy_pred))


print(r2_score(y_test, model1.predict(X_test)))

53/53 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step
0.1067735137752911


In [188]:
md2_data_std = md1_data_std[['B01', 'B05' , 'B08', 'B11','ndwi' , 'ndvi', 'ndbi' , 'UHI Index']]

X2 = md2_data_std.drop(columns=['UHI Index']).values
y2 = md2_data_std['UHI Index'].values


X2_train, X2_test, y2_train, y2_test = train_test_split(X2, y2, test_size=0.2,random_state=123)
print(X_train.shape) 
print(y_test.shape)

(6683, 10)
(1671,)


In [209]:
#model using only the features used in the sample notebook 

model2 = Sequential([
    #Dense(512 , input_shape=(7,), activation='relu'),
    #Dense(128, activation = 'relu'),
    Dense(30, activation = 'relu'),
    Dense(30, activation = 'relu'),
    Dense(30, activation = 'relu'),
    #Dense(64 , activation = 'relu'),
    #Dense(32, activation = 'relu'),
    #Dense(16, activation = 'relu' ),
    Dense(1, activation = 'linear' ) #Output continous variable UHI index 
])


model2.compile(optimizer= 'RMSprop' , loss = "MSE" )

model2.fit(X2_train, y2_train, epochs=25) 

Epoch 1/25
209/209 ━━━━━━━━━━━━━━━━━━━━ 1s 914us/step - loss: 0.9187
Epoch 2/25
209/209 ━━━━━━━━━━━━━━━━━━━━ 0s 856us/step - loss: 0.8657
Epoch 3/25
209/209 ━━━━━━━━━━━━━━━━━━━━ 0s 825us/step - loss: 0.8572
Epoch 4/25
209/209 ━━━━━━━━━━━━━━━━━━━━ 0s 872us/step - loss: 0.8669
Epoch 5/25
209/209 ━━━━━━━━━━━━━━━━━━━━ 0s 851us/step - loss: 0.8390
Epoch 6/25
209/209 ━━━━━━━━━━━━━━━━━━━━ 0s 868us/step - loss: 0.8542
Epoch 7/25
209/209 ━━━━━━━━━━━━━━━━━━━━ 0s 976us/step - loss: 0.8260
Epoch 8/25
209/209 ━━━━━━━━━━━━━━━━━━━━ 0s 826us/step - loss: 0.8535
Epoch 9/25
209/209 ━━━━━━━━━━━━━━━━━━━━ 0s 818us/step - loss: 0.8489
Epoch 10/25
209/209 ━━━━━━━━━━━━━━━━━━━━ 0s 820us/step - loss: 0.8549
Epoch 11/25
209/209 ━━━━━━━━━━━━━━━━━━━━ 0s 832us/step - loss: 0.8709
Epoch 12/25
209/209 ━━━━━━━━━━━━━━━━━━━━ 0s 805us/step - loss: 0.8212
Epoch 13/25
209/209 ━━━━━━━━━━━━━━━━━━━━ 0s 894us/step - loss: 0.8515
Epoch 14/25
209/209 ━━━━━━━━━━━━━━━━━━━━ 0s 861us/step - loss: 0.8206
Epoch 15/25
209/209 ━━━━━━━━━

In [210]:
print(r2_score(y_test, model2.predict(X2_test)))

53/53 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step  
0.13582719515285357


In [211]:
#Model with a regularizer to increas epocs and avoid overfitting

from tensorflow.keras import regularizers



model_lassoreg = Sequential([
    Dense(512 , input_shape=(10,), activation='relu'),
    Dense(128, activation = 'relu' ,
        kernel_regularizer=regularizers.l1(0.01)),
    #Dense(30, activation = 'relu'),
    Dense(64 , activation = 'relu' ,
         kernel_regularizer=regularizers.l1(0.01)),
    Dense(32, activation = 'relu',
         kernel_regularizer=regularizers.l1(0.01)),
    Dense(16, activation = 'relu' ,
         kernel_regularizer=regularizers.l1(0.01)),
    Dense(1, activation = 'linear', 
        kernel_regularizer=regularizers.l1(0.01) ) #Output continous variable UHI index 
])


model_lassoreg.compile(optimizer= 'adam' , loss = "MSE" )

model_lassoreg.fit(X_train, y_train, epochs=100, batch_size=32, validation_split=0.2)


Epoch 1/100
168/168 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 23.0511 - val_loss: 1.8369
Epoch 2/100
168/168 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1.4802 - val_loss: 1.1595
Epoch 3/100
168/168 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1.1341 - val_loss: 1.1205
Epoch 4/100
168/168 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1.1355 - val_loss: 1.1051
Epoch 5/100
168/168 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1.0593 - val_loss: 1.0986
Epoch 6/100
168/168 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1.0857 - val_loss: 1.0954
Epoch 7/100
168/168 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1.1147 - val_loss: 1.0930
Epoch 8/100
168/168 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1.1127 - val_loss: 1.0909
Epoch 9/100
168/168 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1.0908 - val_loss: 1.0909
Epoch 10/100
168/168 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1.0672 - val_loss: 1.0913
Epoch 11/100
168/168 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1.0804 - val_loss: 1.0910
Epoch 12/100
168/168 ━━━━━━━━━━━━━━━━━━━

In [212]:
print(r2_score(y_test, model_lassoreg.predict(X_test)))

53/53 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
-0.004691619025388061


In [217]:
#attempt at helping the network find nonlinear relations
import itertools

md3_data = md1_data_std.copy()
feature_list = md3_data.columns.tolist()

column_combinations = itertools.combinations(feature_list, 2)
print(columns_to_check)
for col1, col2 in column_combinations:
    new_col_name = f'{col1}_{col2}'  # Name for the new column
    md3_data[new_col_name ] = md3_data[col1] * md3_data[col2]  # Multiply the columns


md3_data


KeyboardInterrupt

